A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

In [2]:
import pickle

class SortDataset(Dataset):
    """ 
    Dataset for the Sort problem. E.g. for problem length 6:
    Input: 0 0 2 1 0 1 -> Output: 0 0 0 1 1 2
    Which will feed into the transformer concatenated as:
    input:  0 0 2 1 0 1 0 0 0 1 1
    output: I I I I I 0 0 0 1 1 2
    where I is "ignore", as the transformer is reading the input sequence
    """

    def __init__(self, split, length=6, num_digits=3):
        assert split in {'train', 'test'}
        self.split = split
        self.length = length
        self.num_digits = num_digits
    
    def __len__(self):
        return 10000 # ...
    
    def get_vocab_size(self):
        return self.num_digits
    
    def get_block_size(self):
        # the length of the sequence that will feed into transformer, 
        # containing concatenated input and the output, but -1 because
        # the transformer starts making predictions at the last input element
        return self.length * 2 - 1

    def __getitem__(self, idx):
        
        # use rejection sampling to generate an input example from the desired split
        while True:
            # generate some random integers
            inp = torch.randint(self.num_digits, size=(self.length,), dtype=torch.long)
            # half of the time let's try to boost the number of examples that 
            # have a large number of repeats, as this is what the model seems to struggle
            # with later in training, and they are kind of rate
            if torch.rand(1).item() < 0.5:
                if inp.unique().nelement() > self.length // 2:
                    # too many unqiue digits, re-sample
                    continue
            # figure out if this generated example is train or test based on its hash
            h = hash(pickle.dumps(inp.tolist()))
            inp_split = 'test' if h % 4 == 0 else 'train' # designate 25% of examples as test
            if inp_split == self.split:
                break # ok
        
        # solve the task: i.e. sort
        sol = torch.sort(inp)[0]

        # concatenate the problem specification and the solution
        cat = torch.cat((inp, sol), dim=0)

        # the inputs to the transformer will be the offset sequence
        x = cat[:-1].clone()
        y = cat[1:].clone()
        # we only want to predict at output locations, mask out the loss at the input locations
        y[:self.length-1] = -1
        return x, y


In [12]:
train_dataset[0]

(tensor([0, 1, 2, 0, 2, 1, 0, 0, 1, 1, 2]),
 tensor([-1, -1, -1, -1, -1,  0,  0,  1,  1,  2,  2]))

In [13]:
test_dataset[0]

(tensor([2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2]),
 tensor([-1, -1, -1, -1, -1,  0,  0,  0,  0,  2,  2]))

In [3]:
# print an example instance of the dataset
train_dataset = SortDataset('train')
test_dataset = SortDataset('test')
x, y = train_dataset[0]
for a, b in zip(x,y):
    print(int(a),int(b))

0 -1
1 -1
2 -1
1 -1
2 -1
2 0
0 1
1 1
1 2
2 2
2 2


In [4]:
# create a GPT instance
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

number of parameters: 0.09M


In [5]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 2000
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)

running on device cuda


In [6]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

iter_dt 0.00ms; iter 0: train loss 1.08021
iter_dt 18.20ms; iter 100: train loss 0.13340
iter_dt 18.80ms; iter 200: train loss 0.04775
iter_dt 18.58ms; iter 300: train loss 0.02281
iter_dt 18.75ms; iter 400: train loss 0.06469
iter_dt 18.78ms; iter 500: train loss 0.03846
iter_dt 17.86ms; iter 600: train loss 0.01713
iter_dt 21.15ms; iter 700: train loss 0.01298
iter_dt 18.59ms; iter 800: train loss 0.00182
iter_dt 18.36ms; iter 900: train loss 0.00773
iter_dt 19.15ms; iter 1000: train loss 0.00471
iter_dt 18.19ms; iter 1100: train loss 0.01270
iter_dt 18.24ms; iter 1200: train loss 0.00134
iter_dt 17.76ms; iter 1300: train loss 0.00430
iter_dt 18.08ms; iter 1400: train loss 0.00629
iter_dt 18.55ms; iter 1500: train loss 0.00170
iter_dt 17.99ms; iter 1600: train loss 0.01645
iter_dt 18.20ms; iter 1700: train loss 0.00619
iter_dt 18.14ms; iter 1800: train loss 0.00154
iter_dt 19.38ms; iter 1900: train loss 0.01013


In [14]:
# now let's perform some evaluation
model.eval();

In [15]:
from mingpt.utils import sample

def eval_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        # isolate the input pattern alone
        inp = x[:, :n]
        sol = y[:, -n:]
        # let the model sample the rest of the sequence
        cat = sample(model, inp, n, sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, n:] # isolate the filled in sequence
        # compare the predicted sequence to the true sequence
        correct = (sol == sol_candidate).all(1).cpu() # Software 1.0 vs. Software 2.0 fight RIGHT on this line haha
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            if not correct[i] and mistakes_printed_already < 3: # only print up to 5 mistakes to get a sense
                mistakes_printed_already += 1
                print("GPT claims that %s sorted is %s but gt is %s" % (inp[i].tolist(), sol_candidate[i].tolist(), sol[i].tolist()))
        if max_batches is not None and b+1 >= max_batches:
            break
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_split(trainer, 'train', max_batches=50)
    test_score  = eval_split(trainer, 'test',  max_batches=50)

train final score: 5000/5000 = 100.00% correct
GPT claims that [2, 2, 2, 2, 2, 1] sorted is [2, 2, 2, 2, 2, 2] but gt is [1, 2, 2, 2, 2, 2]
GPT claims that [2, 2, 2, 2, 2, 1] sorted is [2, 2, 2, 2, 2, 2] but gt is [1, 2, 2, 2, 2, 2]
GPT claims that [2, 2, 2, 2, 2, 1] sorted is [2, 2, 2, 2, 2, 2] but gt is [1, 2, 2, 2, 2, 2]
test final score: 4975/5000 = 99.50% correct


In [27]:
# let's run a random given sequence through the model as well
n = train_dataset.length # naugy direct access shrug
inp = torch.tensor([[0, 0, 2, 1, 0, 1]], dtype=torch.long).to(trainer.device)
print(inp)
assert inp[0].nelement() == n
with torch.no_grad():
    cat = sample(model, inp, n, sample=False)
sol = torch.sort(inp[0])[0]
sol_candidate = cat[:, n:]
print('input sequence  :', inp.tolist())
print('predicted sorted:', sol_candidate.tolist())
print('gt sort         :', sol.tolist())
print('matches         :', bool((sol == sol_candidate).all()))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [26]:
with torch.no_grad():
  sample(model, torch.tensor([[1,3,1,4,5,6]], dtype=torch.long).to(trainer.device), 6, sample=False)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.